<div class="alert alert-block alert-success">
<h1>Step 1: modify events in EEG data</h1>
</div>

Author: Hu Chuan-Peng

Affiliation: School of Pyschology, Nanjing Normal Univeristy

Email: hcp4715@hotmail.com

In this notebook, I modified the events data in the EEG data so that the time points when keys are pressed is recorded as events.

In [1]:
%matplotlib inline
import os
import os.path as op
import glob
import mne
import numpy as np
import pandas as pd

import re

from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

In [14]:
root_dir = os.getcwd().split('2015')[0] + '2015' # make sure that the root dir is `moralSelfEEG2015`
print(root_dir)

/media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015


## Load cmobine events file from R-studio and check with one subject's data

We added the RT data in R. which will return a csv file with an additional trigger "resp". Note that we deleted trials without response.

In [3]:
df_ev_new = pd.read_csv('df_events_add_RT.csv')
df_ev_new.head()

,subj_idx,block_id,trial_id,triggers,time,Code
0,6106,1,1,prime_bad_match,8990,32
1,6106,1,1,target_bad_match,10100,5
2,6106,1,1,resp_bad_match,10612,1013
3,6106,1,1,blank_bad_match,11516,999
4,6106,1,2,prime_good_mismatch,11580,48


Again, we started from a single participant's data

In [4]:
events_sub6 = df_ev_new[((df_ev_new['subj_idx'] == 6106) & (df_ev_new['block_id'] == 1)) ]

In [9]:
subj = 'sub6'
cur_raw_dir = op.join(root_dir, "Analysis_Py", "Exp1", 'RawData' , 'eeg' , subj)
cur_file = glob.glob(op.join(cur_raw_dir, 'sub*_1.vhdr'))
print(cur_file)
cur_raw = mne.io.read_raw(cur_file[0])
cur_raw

['/media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub6/sub6_Moral_asso_1.vhdr']
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub6/sub6_Moral_asso_1.vhdr...
Setting channel info structure...


<RawBrainVision | sub6_Moral_asso_1.eeg, 63 x 162150 (324.3 s), ~67 kB, data not loaded>

In [5]:
events_sub6

,subj_idx,block_id,trial_id,triggers,time,Code
0,6106,1,1,prime_bad_match,8990,32
1,6106,1,1,target_bad_match,10100,5
2,6106,1,1,resp_bad_match,10612,1013
3,6106,1,1,blank_bad_match,11516,999
4,6106,1,2,prime_good_mismatch,11580,48
...,...,...,...,...,...,...
451,6106,1,119,blank_neut_mismatch,308286,999
452,6106,1,120,prime_good_mismatch,308350,48
453,6106,1,120,target_good_mismatch,309450,7
454,6106,1,120,resp_good_mismatch,310422,1013


In [15]:
events_new = events_sub6[['time', 'Code', 'triggers']].copy().dropna()
events_new['duration'] = 0
# events_new.rename(columns={"time":"0",
#                            "Code":"2"}, inplace=True) 

# events_new = events_new[["0", "1", "2"]]

events_new.loc[events_new['triggers'].str.contains('prime'), 'duration'] = 50
events_new.loc[events_new['triggers'].str.contains('target'), 'duration'] = 60
events_new.loc[events_new['triggers'].str.contains('blank'), 'duration'] = 50

In [16]:
events_new.head()

,time,Code,triggers,duration
0,8990,32,prime_bad_match,50
1,10100,5,target_bad_match,60
2,10612,1013,resp_bad_match,0
3,11516,999,blank_bad_match,50
4,11580,48,prime_good_mismatch,50


In [22]:
onset = events_new['time'].to_numpy() 
onset = onset/1000.0 # onset should be in seconds.

duration = events_new['duration'].to_numpy()    
duration = duration/1000.0

description = events_new['triggers'].to_numpy()

annotations_new = mne.Annotations(onset, duration, description)  

# cur_raw.set_annotations(annotations_new) 

In [23]:
raw_tmp = cur_raw.copy() # create a temp file
raw_tmp

In [24]:
raw_tmp.set_annotations(annotations_new)

<RawBrainVision | sub6_Moral_asso_1.eeg, 63 x 162150 (324.3 s), ~67 kB, data not loaded>

In [25]:
%matplotlib qt
fig = raw_tmp.plot(start=8, duration=5)  # plot the raw data, it works

Channels marked as bad: none


In [116]:
events_tmp, events_id_tmp =mne.events_from_annotations(raw_tmp,event_id='auto') # we can further check the events get from the annotations.
events_id_tmp

Used Annotations descriptions: ['blank_bad_match', 'blank_bad_mismatch', 'blank_good_match', 'blank_good_mismatch', 'blank_neut_match', 'blank_neut_mismatch', 'prime_bad_match', 'prime_bad_mismatch', 'prime_good_match', 'prime_good_mismatch', 'prime_neut_match', 'prime_neut_mismatch', 'resp_bad_match', 'resp_bad_mismatch', 'resp_good_match', 'resp_good_mismatch', 'resp_neut_match', 'resp_neut_mismatch', 'target_bad_match', 'target_bad_mismatch', 'target_good_match', 'target_good_mismatch', 'target_neut_match', 'target_neut_mismatch']


## Add RT triggers to the EEG dataset

In [26]:
cur_raw_dir = op.join(root_dir, "Analysis_Py", "Exp1", 'RawData' , 'eeg' , "sub*")
cur_raw_dir = glob.glob(cur_raw_dir)
df_list = []

In [29]:
# get the unique block id for each participant who has data in E-Prime
df_ev_new.groupby(['subj_idx','block_id']).size().reset_index().rename(columns={0:'count'})

,subj_idx,block_id,count
0,6106,1,456
1,6106,2,472
2,6106,3,460
3,6106,4,452
4,6106,5,472
...,...,...,...
202,6131,6,476
203,6131,7,468
204,6131,8,440
205,6131,9,448


In [78]:
raw_dir = op.join(root_dir, "Analysis_Py", "Exp1", 'RawData' , 'eeg')
fif_dir = op.join(root_dir, "Analysis_Py", "Exp1", 'raw_fif_new')
raw_dir

'/media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg'

In [81]:
sub_folders = [name for name in os.listdir(raw_dir) if os.path.isdir(os.path.join(raw_dir, name))]
# sub_folders
for dir in sub_folders:
#     print(dir)
    subj_idx = int(dir.partition("sub")[2]) + 6100
#     print(subj_idx)
    cur_files = glob.glob(op.join(raw_dir, dir, 'sub*.vhdr'))
#     print(len(cur_files))
    if subj_idx in df_ev_new.subj_idx.unique():
        # print("yes")
        cur_df_ev = df_ev_new.loc[df_ev_new['subj_idx'] == subj_idx]
        
        # make sure that all participants has all block, exlcude participant 6119 and 6124
        if len(cur_df_ev.block_id.unique()) == len(cur_files):
            # print("subj: ", subj_idx,len(cur_files))
            for cur_file in cur_files:
                # print(cur_file)
                block_id = int(re.findall(r'asso_(.*?)\.vhdr', cur_file)[0])
                print("working on subj: ", subj_idx, "block_id: ", block_id)

                cur_raw = mne.io.read_raw(cur_file)

                events_new = cur_df_ev[['time', 'Code', 'triggers']].copy().dropna()
                events_new['duration'] = 0
                events_new.loc[events_new['triggers'].str.contains('prime'), 'duration']  = 50
                events_new.loc[events_new['triggers'].str.contains('target'), 'duration'] = 60
                events_new.loc[events_new['triggers'].str.contains('blank'), 'duration']  = 50

                onset = events_new['time'].to_numpy() 
                onset = onset/1000.0 # onset should be in seconds.

                duration = events_new['duration'].to_numpy()    
                duration = duration/1000.0

                description = events_new['triggers'].to_numpy()
                annotations_new = mne.Annotations(onset, duration, description) 

                cur_raw.set_annotations(annotations_new)
                
                if (block_id == 1):
                    raw = cur_raw.copy()
                else:
                    mne.io.Raw.append(self=raw, raws=cur_raw)
                    
            raw.save(fif_dir + "/" + str(subj_idx) +'_raw.fif',overwrite=True)

        else:
            continue
        # print(cur_files)

    else:
        # print(subj_idx)
        continue

#     for file in cur_files:
#         print(file)
#         subj_idx = int(re.findall(r'sub(.*?)\/sub', file)[0]) + 6100
#         block_id = int(re.findall(r'asso_(.*?)\.vhdr', file)[0])
#         # print("subj_idx is: ", subj_idx, "; block_id is: ", block_id, ".")
#         # if (subj_idx in df_ev_new.subj_idx.unique()): 
#         cur_df_ev = df_ev_new.loc[df_ev_new['subj_idx'] == subj_idx]
#         # if (block_id in cur_df_ev.block_id.unique()):
#         #     print("yes")


        # else:
        #     print("block", block_id, "of subj", subj_idx, "is skipped for now") # three participants are skipped
        #     continue


working on subj:  6110 block_id:  1
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_1.vhdr...
Setting channel info structure...
working on subj:  6110 block_id:  2
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_2.vhdr...
Setting channel info structure...
working on subj:  6110 block_id:  3
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_3.vhdr...
Setting channel info structure...


/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 45 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)
/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 37 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)
/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 43 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)


working on subj:  6110 block_id:  4
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_4.vhdr...
Setting channel info structure...
working on subj:  6110 block_id:  5
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_5.vhdr...
Setting channel info structure...
working on subj:  6110 block_id:  6
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_6.vhdr...
Setting channel info structure...


/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 43 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)
/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 37 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)


working on subj:  6110 block_id:  7
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub10/sub10_Moral_asso_7.vhdr...
Setting channel info structure...


...

working on subj:  6108 block_id:  6
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub8/sub8_Moral_asso_6.vhdr...
Setting channel info structure...
working on subj:  6108 block_id:  7
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub8/sub8_Moral_asso_7.vhdr...
Setting channel info structure...
working on subj:  6108 block_id:  8
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub8/sub8_Moral_asso_8.vhdr...
Setting channel info structure...
working on subj:  6108 block_id:  9
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub8/sub8_Moral_asso_9.vhdr...
Setting channel info structure...
Writing /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/raw_fif_new/6108_raw.fif
Closing /media/hcp4715/RAID32T/Data_o

/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 3 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)
/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 5 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)


working on subj:  6109 block_id:  4
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub9/sub9_Moral_asso_4.vhdr...
Setting channel info structure...
working on subj:  6109 block_id:  5
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub9/sub9_Moral_asso_5.vhdr...
Setting channel info structure...


/tmp/ipykernel_929370/15992950.py:38: RuntimeWarning: Omitted 5 annotation(s) that were outside data range.
  cur_raw.set_annotations(annotations_new)


working on subj:  6109 block_id:  6
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub9/sub9_Moral_asso_6.vhdr...
Setting channel info structure...
working on subj:  6109 block_id:  7
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub9/sub9_Moral_asso_7.vhdr...
Setting channel info structure...
working on subj:  6109 block_id:  8
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub9/sub9_Moral_asso_8.vhdr...
Setting channel info structure...
working on subj:  6109 block_id:  9
Extracting parameters from /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/RawData/eeg/sub9/sub9_Moral_asso_9.vhdr...
Setting channel info structure...
Writing /media/hcp4715/RAID32T/Data_own/eegData/moralSelfEEG2015/Analysis_Py/Exp1/raw_fif_new/6109_raw.fif
Closing /media/hcp4715/RAID32T/Data_o

In [ ]:
raw = mne.io.read_raw_brainvision(block1_fname, montage, preload=True)# read .vhdr as mne raw instance
for i in range(2,10):#read block 2 to block9  and append them to block1
    block_path = (subject +'_Moral_asso_'+str(i)+'.vhdr')
    print(block_path)
    
    eeg_path = os.path.join(raw_name_path, block_path)
    
    raw1 = mne.io.read_raw_brainvision(eeg_path, montage, preload=True)
    
    mne.io.Raw.append(self=raw, raws=raw1)

if len(subject) == 4:  #change the subject name from sub6 to sub06 and save them as fif
    d = list(subject)
    d.insert(3,'0')
    subject1 = "".join(d)
    raw.save('raw_fif/'+subject1+'_raw.fif',overwrite=True)
elif len(subject) == 5:
    raw.save('raw_fif/'+subject1+'_raw.fif',overwrite=True)

In [19]:
int(re.findall(r'asso_(.*?)\.vhdr', file)[0])

2

In [17]:
re.search('sub(.+?)\/sub', file).group(1)

'21'